In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
#import torch.utils.tensorboard as tb
from Preprocessing.preprocessing import PreprocessingTrainingData
from torch.utils.data.sampler import SequentialSampler
import seaborn as sns
import matplotlib.pyplot as  plt
import os
import logging
torch.set_printoptions(profile="full")
import pandas as pd
from Postprocessing.postprocessing import PostProcessing

In [2]:
#static parameters
train_batch_size = 170
val_batch_size = 170
sequence_length=50
test_batch_size = 1
input_size = 1
hidden_size = 256
num_layer = 2
output_size = 38
clip = 3

In [3]:
#get data from preprocessing.py
dataset_path = os.path.join(os.path.abspath('..'),'Dataset\\Clementi dataset\\Clementi dataset' )
network_input,network_output,max_midi_number,min_midi_number,int_to_note = PreprocessingTrainingData().preprocess_notes(dataset_path)
network_input, network_output = network_input.cuda(), network_output.cuda()

# print(network_input)
#print(network_output)

In [4]:
print(network_output.max())
print(network_output.min())

tensor(37, device='cuda:0')
tensor(0, device='cuda:0')


In [5]:
#network_output

In [6]:
print(network_input.max())
print(network_input.min())
print(max_midi_number)
print(min_midi_number)
print(int_to_note)

tensor(1., device='cuda:0')
tensor(0., device='cuda:0')
89
50
{0: 50, 1: 52, 2: 53, 3: 54, 4: 55, 5: 56, 6: 57, 7: 58, 8: 59, 9: 60, 10: 61, 11: 62, 12: 63, 13: 64, 14: 65, 15: 66, 16: 67, 17: 68, 18: 69, 19: 70, 20: 71, 21: 72, 22: 73, 23: 74, 24: 75, 25: 76, 26: 77, 27: 78, 28: 79, 29: 80, 30: 81, 31: 82, 32: 83, 33: 84, 34: 85, 35: 86, 36: 88, 37: 89}


In [7]:
'''
data is highly unbalanced
# '''
# sns.distplot(torch.tensor(network_output).cpu())
# xx = pd.DataFrame(torch.tensor(network_output).cpu())
# xx.groupby(0).size().to_frame(name='values')

'\ndata is highly unbalanced\n# '

In [8]:
'''
to make batch of equal sizes
Quick Fix
'''
network_input = network_input[: -117]
network_output = network_output[: -117]

print(network_input.shape)
print(network_output.shape)

torch.Size([8500, 50, 1])
torch.Size([8500])


In [9]:
# # Bakchodi Normalization
# network_input=network_input.cpu().numpy().tolist()
# for i in range(len(network_input)):
#     for j in range(len(network_input[i])):
#         network_input[i][j][0]=((network_input[i][j][0])*(max_midi_number-min_midi_number)+min_midi_number)/max_midi_number
# network_input=torch.Tensor(network_input).cuda()

In [10]:
#network_input[:10]

In [11]:
'''
create Stacked LSTM model
'''
class Stacked_LSTM(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers, output_size):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        self.lstm1 = nn.LSTM(input_size = input_size, hidden_size = hidden_size, batch_first = True)
        self.lstm2 = nn.LSTM(input_size = hidden_size, hidden_size = output_size,batch_first = True)
        
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(output_size, output_size)
        
        
    def forward(self, x, batch_size):
        
        output, _ = self.lstm1(x)        
        #print(output.shape)
        
        #output = self.dropout(output)
        
        output, _ = self.lstm2(output)
        #print(output.shape)
        
        # stack up lstm outputs
        output = output.contiguous().view(-1, 38)
        #print(output.shape)
        
        output = self.dropout(output)
        output = self.linear(output)
        #print('Linear Output :-',output.shape)
        
        #output = F.softmax(output, dim = 1)
        #print('SOFTMAX OUTPUT :--', output)
        
        
        # reshape to be batch_size first
        output = output.view(batch_size, -1)
        #print('Reshape to batch size first :-',output.shape)
        
        output = output[:, -self.output_size:] # get last batch of labels
        #print('Final Output :-',output)
        #print('RESHAPE SIZE :-', output.shape)
        
        return output

#initialize the weights of LSTM using Xavier initialization    
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight) 

In [12]:
'''
Divide the dataset into train/val 
'''
train_size = 0.8
indices = list(range(len(network_input)))
split = int(np.floor(train_size*len(network_input)))
train_idx, val_idx = indices[:split], indices[split:]

train_sampler = SequentialSampler(train_idx)
val_sampler = SequentialSampler(val_idx)

dataset = TensorDataset(network_input,network_output)
train_loader = DataLoader(dataset, batch_size= train_batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size= val_batch_size,sampler= val_sampler)

In [13]:
import torch.optim as optimizer

model = Stacked_LSTM(input_size,hidden_size,num_layer,output_size)
model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optimizer.AdamW(model.parameters())
#optimizer = optimizer.RMSprop(model.parameters(), lr = 0.001, weight_decay = 0.01)


#make sure to transfer model to GPU after initializing optimizer
model.cuda()

Stacked_LSTM(
  (lstm1): LSTM(1, 256, batch_first=True)
  (lstm2): LSTM(256, 38, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (linear): Linear(in_features=38, out_features=38, bias=True)
)

In [14]:
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.01
)

In [15]:
epochs = 500
min_val_loss = np.Inf

for e in range(epochs):
    
    train_loss = 0
    val_loss = 0
    train_accuracy = 0
    val_accuracy = 0
    
    for inputs,labels in train_loader:
        #print(inputs.shape)

        '''
        Creating new variables for the hidden state, otherwise
        we'd backprop through the entire training history
        '''
        
        # zero accumulated gradients
        model.zero_grad()
       
        # get the output from the model
        output = model.forward(inputs, train_batch_size)
        #print('OUTPUT', output)
        
        
        #print('Labels Shape :-', (torch.max(labels, 1)[1]).shape)
    
        # calculate the loss and perform backprop
        #print('Labels Long :-', labels.long())
        loss = criterion(output,labels.long())
        #print('LOSS IS :-', loss)
        
        loss.backward()
        
        #calculate training accuracy
        output = F.softmax(output, dim = 1)
        top_p, top_class = output.topk(1, dim=1)
        #logging.debug(' top probab {} top class {}'.format(top_p.view(-1, top_p.shape[0]), top_class.view(-1, top_p.shape[0])))

        equals = top_class == labels.long().view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        #nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        train_loss += loss.item()
        #print(train_loss)
              
    model.eval()
    for inputs, labels in val_loader:
                
        output = model.forward(inputs, val_batch_size)
       
        loss = criterion(output,labels.long())
        
        val_loss += loss.item()
        
        #calculate validation accuracy
        output = F.softmax(output, dim = 1)
        top_p, top_class = output.topk(1, dim=1)
        
        #logging.debug(output)
        #logging.debug('VALIDATION top probab {} VALIDATION top class {}'.format(top_p.view(-1, top_p.shape[0]), top_class.view(-1, top_p.shape[0])))

        #print('Top Class:- ',top_class)
        equals = top_class == labels.long().view(*top_class.shape)
        #print('Equals:- ', equals)
        val_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    model.train()
    
    #Averaging losses
    train_loss = train_loss/len(train_loader)
    val_loss = val_loss/len(val_loader)
    val_accuracy = val_accuracy/len(val_loader)
    train_accuracy = train_accuracy/len(train_loader)
    
    print('Epoch: {}\tTrain Loss: {:.7f} \tVal Loss:{:.7f} \tTrain Acc: {:.7}% \tVal Acc: {:.7f}%'.format(e, train_loss, val_loss, train_accuracy*100,val_accuracy*100))
    
    #saving the model if validation loss is decreased
    if val_loss <= min_val_loss:
        print('Validation Loss decreased from {:6f} to {:6f}, saving the model weights'.format(min_val_loss, val_loss))
        torch.save(model.state_dict(), 'lstm_state_256-38-bak_norm.pt')
        min_val_loss = val_loss

Epoch: 0	Train Loss: 3.3668325 	Val Loss:3.0439595 	Train Acc: 8.352941% 	Val Acc: 11.4117651%
Validation Loss decreased from    inf to 3.043960, saving the model weights
Epoch: 1	Train Loss: 3.1634846 	Val Loss:2.9785109 	Train Acc: 9.029412% 	Val Acc: 11.4117651%
Validation Loss decreased from 3.043960 to 2.978511, saving the model weights
Epoch: 2	Train Loss: 3.1245804 	Val Loss:2.9572137 	Train Acc: 8.764706% 	Val Acc: 11.4117651%
Validation Loss decreased from 2.978511 to 2.957214, saving the model weights
Epoch: 3	Train Loss: 3.1104905 	Val Loss:2.9583088 	Train Acc: 9.279412% 	Val Acc: 11.4117651%
Epoch: 4	Train Loss: 3.0969946 	Val Loss:2.9501348 	Train Acc: 8.647059% 	Val Acc: 11.4117651%
Validation Loss decreased from 2.957214 to 2.950135, saving the model weights
Epoch: 5	Train Loss: 3.0861625 	Val Loss:2.9469243 	Train Acc: 9.161765% 	Val Acc: 11.4117651%
Validation Loss decreased from 2.950135 to 2.946924, saving the model weights
Epoch: 6	Train Loss: 3.0828948 	Val Loss:2

Epoch: 62	Train Loss: 2.8406813 	Val Loss:2.7188312 	Train Acc: 13.10294% 	Val Acc: 12.4705886%
Validation Loss decreased from 2.801621 to 2.718831, saving the model weights
Epoch: 63	Train Loss: 2.8018362 	Val Loss:2.6632511 	Train Acc: 13.25% 	Val Acc: 14.0588240%
Validation Loss decreased from 2.718831 to 2.663251, saving the model weights
Epoch: 64	Train Loss: 2.7548085 	Val Loss:2.6274626 	Train Acc: 14.36765% 	Val Acc: 13.7647063%
Validation Loss decreased from 2.663251 to 2.627463, saving the model weights
Epoch: 65	Train Loss: 2.7029635 	Val Loss:2.5922425 	Train Acc: 14.61765% 	Val Acc: 13.2352944%
Validation Loss decreased from 2.627463 to 2.592242, saving the model weights
Epoch: 66	Train Loss: 2.6538521 	Val Loss:2.5720674 	Train Acc: 15.69118% 	Val Acc: 13.2941179%
Validation Loss decreased from 2.592242 to 2.572067, saving the model weights
Epoch: 67	Train Loss: 2.6302821 	Val Loss:2.5501688 	Train Acc: 15.77941% 	Val Acc: 12.4705885%
Validation Loss decreased from 2.5720

Epoch: 120	Train Loss: 2.3556506 	Val Loss:2.3186349 	Train Acc: 21.33824% 	Val Acc: 22.1176475%
Validation Loss decreased from 2.325791 to 2.318635, saving the model weights
Epoch: 121	Train Loss: 2.3546812 	Val Loss:2.3189863 	Train Acc: 21.63235% 	Val Acc: 20.9411769%
Epoch: 122	Train Loss: 2.3534755 	Val Loss:2.3123602 	Train Acc: 21.44118% 	Val Acc: 21.4705887%
Validation Loss decreased from 2.318635 to 2.312360, saving the model weights
Epoch: 123	Train Loss: 2.3429385 	Val Loss:2.3108058 	Train Acc: 21.58824% 	Val Acc: 21.8235299%
Validation Loss decreased from 2.312360 to 2.310806, saving the model weights
Epoch: 124	Train Loss: 2.3343522 	Val Loss:2.3071954 	Train Acc: 21.79412% 	Val Acc: 21.8823534%
Validation Loss decreased from 2.310806 to 2.307195, saving the model weights
Epoch: 125	Train Loss: 2.3414739 	Val Loss:2.3289087 	Train Acc: 21.89706% 	Val Acc: 23.6470592%
Epoch: 126	Train Loss: 2.3664137 	Val Loss:2.3085016 	Train Acc: 20.51471% 	Val Acc: 22.0588239%
Epoch: 12

Epoch: 184	Train Loss: 2.2410796 	Val Loss:2.2361085 	Train Acc: 25.58824% 	Val Acc: 23.7647066%
Epoch: 185	Train Loss: 2.2384618 	Val Loss:2.2327602 	Train Acc: 25.82353% 	Val Acc: 23.6470596%
Epoch: 186	Train Loss: 2.2335426 	Val Loss:2.1713254 	Train Acc: 27.30882% 	Val Acc: 26.8823539%
Validation Loss decreased from 2.178463 to 2.171325, saving the model weights
Epoch: 187	Train Loss: 2.2043098 	Val Loss:2.1713372 	Train Acc: 27.44118% 	Val Acc: 26.4705889%
Epoch: 188	Train Loss: 2.1760639 	Val Loss:2.1694090 	Train Acc: 27.61765% 	Val Acc: 26.3529418%
Validation Loss decreased from 2.171325 to 2.169409, saving the model weights
Epoch: 189	Train Loss: 2.1913618 	Val Loss:2.1860462 	Train Acc: 27.64706% 	Val Acc: 25.4117654%
Epoch: 190	Train Loss: 2.1841600 	Val Loss:2.1711944 	Train Acc: 28.10294% 	Val Acc: 26.1176480%
Epoch: 191	Train Loss: 2.1876867 	Val Loss:2.1693578 	Train Acc: 27.54412% 	Val Acc: 27.7647066%
Validation Loss decreased from 2.169409 to 2.169358, saving the mode

Epoch: 256	Train Loss: 2.0713815 	Val Loss:2.2584716 	Train Acc: 32.75% 	Val Acc: 25.1176478%
Epoch: 257	Train Loss: 2.1694637 	Val Loss:2.4134885 	Train Acc: 28.73529% 	Val Acc: 22.0588242%
Epoch: 258	Train Loss: 2.1694488 	Val Loss:2.1912389 	Train Acc: 28.73529% 	Val Acc: 28.3529419%
Epoch: 259	Train Loss: 2.1216606 	Val Loss:2.2268994 	Train Acc: 30.73529% 	Val Acc: 27.1176478%
Epoch: 260	Train Loss: 2.0943689 	Val Loss:2.0901474 	Train Acc: 31.54412% 	Val Acc: 30.8823538%
Epoch: 261	Train Loss: 2.0549412 	Val Loss:2.0667582 	Train Acc: 32.57353% 	Val Acc: 30.1764715%
Epoch: 262	Train Loss: 2.0385152 	Val Loss:2.0937021 	Train Acc: 33.39706% 	Val Acc: 29.6470597%
Epoch: 263	Train Loss: 2.0153606 	Val Loss:2.0790850 	Train Acc: 33.69118% 	Val Acc: 30.8823539%
Epoch: 264	Train Loss: 2.0222410 	Val Loss:2.0668973 	Train Acc: 33.92647% 	Val Acc: 31.1764717%
Epoch: 265	Train Loss: 2.0392090 	Val Loss:2.1804308 	Train Acc: 32.82353% 	Val Acc: 25.4705890%
Epoch: 266	Train Loss: 2.0249967 

Epoch: 327	Train Loss: 1.7175130 	Val Loss:1.6689204 	Train Acc: 45.23529% 	Val Acc: 47.0588246%
Epoch: 328	Train Loss: 1.7319995 	Val Loss:1.6269247 	Train Acc: 43.95588% 	Val Acc: 49.4705889%
Epoch: 329	Train Loss: 1.7197523 	Val Loss:1.6735374 	Train Acc: 44.38235% 	Val Acc: 46.2352943%
Epoch: 330	Train Loss: 1.7074203 	Val Loss:1.6008882 	Train Acc: 44.66177% 	Val Acc: 49.4117659%
Validation Loss decreased from 1.611160 to 1.600888, saving the model weights
Epoch: 331	Train Loss: 1.7235635 	Val Loss:1.6465123 	Train Acc: 44.54412% 	Val Acc: 47.3529413%
Epoch: 332	Train Loss: 1.6991572 	Val Loss:1.6973015 	Train Acc: 44.48529% 	Val Acc: 46.3529417%
Epoch: 333	Train Loss: 1.7224483 	Val Loss:1.7452251 	Train Acc: 44.2353% 	Val Acc: 44.0588242%
Epoch: 334	Train Loss: 1.7440221 	Val Loss:1.6547145 	Train Acc: 43.66177% 	Val Acc: 46.2352946%
Epoch: 335	Train Loss: 1.7095190 	Val Loss:1.6169124 	Train Acc: 44.69118% 	Val Acc: 48.5882363%
Epoch: 336	Train Loss: 1.6705147 	Val Loss:1.62970

Epoch: 401	Train Loss: 1.3581283 	Val Loss:1.1612062 	Train Acc: 58.33824% 	Val Acc: 66.1764711%
Validation Loss decreased from 1.230336 to 1.161206, saving the model weights
Epoch: 402	Train Loss: 1.3093110 	Val Loss:1.1238968 	Train Acc: 59.47059% 	Val Acc: 67.9411763%
Validation Loss decreased from 1.161206 to 1.123897, saving the model weights
Epoch: 403	Train Loss: 1.2859181 	Val Loss:1.1105361 	Train Acc: 59.67647% 	Val Acc: 67.0588243%
Validation Loss decreased from 1.123897 to 1.110536, saving the model weights
Epoch: 404	Train Loss: 1.2964725 	Val Loss:1.1004819 	Train Acc: 59.54412% 	Val Acc: 67.7647066%
Validation Loss decreased from 1.110536 to 1.100482, saving the model weights
Epoch: 405	Train Loss: 1.2901407 	Val Loss:1.0674929 	Train Acc: 58.95588% 	Val Acc: 70.5882353%
Validation Loss decreased from 1.100482 to 1.067493, saving the model weights
Epoch: 406	Train Loss: 1.2923778 	Val Loss:1.0863791 	Train Acc: 58.55882% 	Val Acc: 69.1176474%
Epoch: 407	Train Loss: 1.254

Epoch: 472	Train Loss: 0.9523204 	Val Loss:0.7758206 	Train Acc: 70.16176% 	Val Acc: 77.8235292%
Epoch: 473	Train Loss: 0.9524773 	Val Loss:0.7378816 	Train Acc: 69.69118% 	Val Acc: 78.4705883%
Validation Loss decreased from 0.768804 to 0.737882, saving the model weights
Epoch: 474	Train Loss: 0.9763717 	Val Loss:0.8041264 	Train Acc: 69.27941% 	Val Acc: 75.4705876%
Epoch: 475	Train Loss: 0.9606206 	Val Loss:0.7999308 	Train Acc: 70.23529% 	Val Acc: 76.2352943%
Epoch: 476	Train Loss: 0.9671556 	Val Loss:0.8165788 	Train Acc: 69.20588% 	Val Acc: 75.3529412%
Epoch: 477	Train Loss: 0.9453627 	Val Loss:0.7550468 	Train Acc: 70.36765% 	Val Acc: 77.8823525%
Epoch: 478	Train Loss: 0.9170957 	Val Loss:0.7164104 	Train Acc: 71.41176% 	Val Acc: 78.6470586%
Validation Loss decreased from 0.737882 to 0.716410, saving the model weights
Epoch: 479	Train Loss: 0.9077663 	Val Loss:0.7715735 	Train Acc: 72.23529% 	Val Acc: 77.1176463%
Epoch: 480	Train Loss: 0.9096650 	Val Loss:0.8117018 	Train Acc: 71.

## MUSIC GENERATION

In [ ]:
#load weights
test_model = Stacked_LSTM(input_size,hidden_size,num_layer,output_size)
test_model.load_state_dict(torch.load('lstm_state_256-38-our.pt'))
test_model.eval()
test_model.cuda()

In [ ]:
#load population database
#testing_data = np.ones(200)*1
testing_data = list(range(50,90))
testing_data.extend(testing_data[::-1])
testing_data_rev = testing_data[::-1]
testing_data_rev.extend(testing_data)
testing_data_rev.extend(testing_data_rev)
testing_data = testing_data_rev


testing_data = np.asarray(testing_data)
testing_data = testing_data.reshape(testing_data.shape[0],1)

initial_seq = [network_input[0][1:].cpu().numpy().tolist()]

testing_data_unnorm = testing_data

In [ ]:
#abc

In [ ]:
list1=[]
testing_data=testing_data.tolist()
for i in range(len(testing_data)):
    list1.extend(testing_data[i])

#list1

for i in range(len(list1)):
    list1[i]=(list1[i]-50)/(89-50)
#     list1[i]=(list1[i])/(89)

list1 = np.asarray(list1)
list1 = list1.reshape(list1.shape[0],1)
testing_data = list1
#list1

In [ ]:
#testing_data

In [ ]:
#Predicting
def prediction_with_influence(influence,int2note,initial_seq, max_note, min_note,test_batch_size = 1):

    predicted_notes = []
    initial_seq[0].extend([[0]]*len(testing_data))
    test_seq = torch.Tensor(initial_seq).cuda()
    
    for i in range(len(influence)):
        
        test_seq[0][sequence_length - 1 + i][0] = float(influence[i])
        
        test_slice = test_seq[0][i : i + sequence_length]        
        test_slice = test_slice.view(1, test_slice.shape[0], test_slice.shape[1])
        
        test_output = test_model.forward(test_slice, test_batch_size)
    
        test_output = F.softmax(test_output, dim = 1)
        top_p, top_class = test_output.topk(1,dim =1)
        
        test_seq[0][sequence_length - 1 + i][0] = (int2note[top_class.item()] - min_note)/(max_note - min_note)
#         test_seq[0][sequence_length - 1 + i][0] = int2note[top_class.item()]/max_note
        
        predicted_notes.append(int2note[top_class.item()])
        
    return predicted_notes

In [ ]:
predicted_notes_lst = prediction_with_influence(testing_data,int_to_note,initial_seq, max_midi_number, min_midi_number)

In [ ]:
#predicted_notes_lst

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#plt.plot(list(np.ones(200)*89))

#plt.plot(list(np.ones(200)*50))
#plt.plot(list(np.ones(20)*50))
plt.plot(testing_data_unnorm)
plt.plot(predicted_notes_lst)

In [ ]:
from collections import Counter
c = Counter(predicted_notes_lst)
c

In [ ]:
PostProcessing().generate_midi_file('hello.midi', predicted_notes_lst)